In [1]:
import sys
sys.path.append('..')
import config
from fqc import uccsd, qaoa, util

import numpy as np
from datetime import datetime

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [2]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [3]:
def binary_search_for_shortest_pulse_time(min_time, max_time, convergence, reg_coeffs):
    """Search between [min_time, max_time] up to 1ns tolerance. Assumes 20 steps per ns.""" 
    min_steps, max_steps = min_time * 20, max_time * 20
    while min_steps + 20 < max_steps: # just estimate to +- 1ns
        mid_steps = int((min_steps + max_steps) / 2)
        total_time = mid_steps / 20.0
        print('\n\ntrying total_time: %s for unitary of size %s' % (str(total_time), str(U.shape)))
        SS = Grape(H0, Hops, Hnames, U, total_time, mid_steps, states_concerned_list, convergence,
                    reg_coeffs=reg_coeffs,
                    use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA, show_plots=False, file_name=file_name, data_path=data_path)
        if SS.l < SS.conv.conv_target: # if converged, search lower half 
            max_steps = mid_steps
        else:
            min_steps = mid_steps
    return mid_steps / 20

## 3Reg Full

In [ ]:
full_times = []
for N in range(4, 5, 2):
    row = 2
    col = int(N / 2)
    d = 2 # this is the number of energy levels to consider (i.e. d-level qudits) 
    max_iterations = (2**N) * 125
    decay = max_iterations / 2
    convergence = {'rate':0.01, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-9, 'update_step': 20}
    reg_coeffs = {}
    for p in range(1,7):
        circuit = qaoa.get_qaoa_circuit(N, p, '3Reg')
        # TODO: average over 10 graphs
        U = util.circuitutil.get_unitary(circuit)
        connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(row, col, directed=False)
        H0 = hamiltonian.get_H0(N, d)
        Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
        states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
        maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)
        shortest_time = binary_search_for_shortest_pulse_time(10, int(90*(N/5)*p), convergence, reg_coeffs)
        full_times.append(shortest_time)



trying total_time: 41.0 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/yongshan/00006_Mar13.h5
Using 5 Taylor terms and 4 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.78e-01; Runtime: 2.5s; Iterations = 0, grads =   2.068e-02, unitary_metric = 1.00049
Error = :5.78e-01; Runtime: 35.2s; Iterations = 20, grads =   9.584e-01, unitary_metric = 1.00028
Error = :3.99e-01; Runtime: 63.1s; Iterations = 40, grads =   2.401e-01, unitary_metric = 1.00044
Error = :3.76e-01; Runtime: 91.1s; Iterations = 60, grads =   2.634e-02, unitary_metric = 1.00045
Error = :3.74e-01; Runtime: 119.0s; Iterations = 80, grads =   4.331e-03, unitary_metric = 1.00041
Error = :3.80e-01; Runtime: 147.1s; Iterations = 100, grads =   2.56

Error = :3.74e-01; Runtime: 162.5s; Iterations = 140, grads =   2.493e-02, unitary_metric = 1.00018
Error = :3.74e-01; Runtime: 184.8s; Iterations = 160, grads =   3.606e-05, unitary_metric = 1.00018
Error = :3.74e-01; Runtime: 207.0s; Iterations = 180, grads =   3.280e-03, unitary_metric = 1.00018
Error = :3.74e-01; Runtime: 229.3s; Iterations = 200, grads =   2.625e-02, unitary_metric = 1.00018
Error = :3.74e-01; Runtime: 253.1s; Iterations = 220, grads =   1.635e-03, unitary_metric = 1.00019
Error = :3.74e-01; Runtime: 275.4s; Iterations = 240, grads =   2.105e-04, unitary_metric = 1.00020
Error = :3.76e-01; Runtime: 297.6s; Iterations = 260, grads =   6.421e-02, unitary_metric = 1.00019
Error = :3.74e-01; Runtime: 319.9s; Iterations = 280, grads =   9.416e-04, unitary_metric = 1.00019
Error = :3.74e-01; Runtime: 342.2s; Iterations = 300, grads =   3.411e-04, unitary_metric = 1.00019
Error = :3.76e-01; Runtime: 366.0s; Iterations = 320, grads =   6.080e-02, unitary_metric = 1.00019


Error = :3.74e-01; Runtime: 713.3s; Iterations = 760, grads =   7.076e-07, unitary_metric = 1.00002
Error = :3.74e-01; Runtime: 731.8s; Iterations = 780, grads =   1.290e-08, unitary_metric = 1.00002
Error = :3.74e-01; Runtime: 750.2s; Iterations = 800, grads =   8.100e-09, unitary_metric = 1.00002
Error = :3.74e-01; Runtime: 764.4s; Iterations = 814, grads =   8.923e-10, unitary_metric = 1.00002
data saved at: /project/ftchong/qoc/yongshan/00010_Mar13.h5


trying total_time: 28.35 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/yongshan/00011_Mar13.h5
Using 7 Taylor terms and 2 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.86e-01; Runtime: 1.7s; Iterations = 0, grads =   1.255e-02, unitary_metric = 1.00003


Error = :1.44e-03; Runtime: 3070.0s; Iterations = 1080, grads =   9.211e-07, unitary_metric = 1.00042
Error = :1.40e-03; Runtime: 3125.7s; Iterations = 1100, grads =   8.380e-07, unitary_metric = 1.00039
Error = :1.36e-03; Runtime: 3185.4s; Iterations = 1120, grads =   7.695e-07, unitary_metric = 1.00039
Error = :1.30e-03; Runtime: 3241.1s; Iterations = 1140, grads =   7.143e-07, unitary_metric = 1.00042
Error = :1.26e-03; Runtime: 3297.0s; Iterations = 1160, grads =   6.636e-07, unitary_metric = 1.00041
Error = :1.22e-03; Runtime: 3352.7s; Iterations = 1180, grads =   6.263e-07, unitary_metric = 1.00042
Error = :1.19e-03; Runtime: 3408.7s; Iterations = 1200, grads =   5.946e-07, unitary_metric = 1.00042
Error = :1.15e-03; Runtime: 3468.0s; Iterations = 1220, grads =   5.614e-07, unitary_metric = 1.00043
Error = :1.12e-03; Runtime: 3524.0s; Iterations = 1240, grads =   5.388e-07, unitary_metric = 1.00042
Error = :1.09e-03; Runtime: 3579.7s; Iterations = 1260, grads =   5.158e-07, unita

Error = :1.52e-02; Runtime: 1855.9s; Iterations = 1240, grads =   3.323e-06, unitary_metric = 1.00022
Error = :1.50e-02; Runtime: 1885.4s; Iterations = 1260, grads =   3.050e-06, unitary_metric = 1.00023
Error = :1.48e-02; Runtime: 1914.9s; Iterations = 1280, grads =   3.365e-06, unitary_metric = 1.00023
Error = :1.45e-02; Runtime: 1944.2s; Iterations = 1300, grads =   4.348e-06, unitary_metric = 1.00023
Error = :1.43e-02; Runtime: 1975.5s; Iterations = 1320, grads =   6.070e-06, unitary_metric = 1.00022
Error = :1.40e-02; Runtime: 2004.8s; Iterations = 1340, grads =   8.490e-06, unitary_metric = 1.00020
Error = :1.35e-02; Runtime: 2034.2s; Iterations = 1360, grads =   1.117e-05, unitary_metric = 1.00019
Error = :1.29e-02; Runtime: 2063.5s; Iterations = 1380, grads =   1.342e-05, unitary_metric = 1.00019
Error = :1.22e-02; Runtime: 2092.8s; Iterations = 1400, grads =   1.415e-05, unitary_metric = 1.00020
Error = :1.15e-02; Runtime: 2124.1s; Iterations = 1420, grads =   1.231e-05, unita

Error = :4.79e-03; Runtime: 1726.4s; Iterations = 760, grads =   3.422e-06, unitary_metric = 1.00027
Error = :4.60e-03; Runtime: 1773.5s; Iterations = 780, grads =   3.458e-06, unitary_metric = 1.00027
Error = :4.41e-03; Runtime: 1820.5s; Iterations = 800, grads =   3.468e-06, unitary_metric = 1.00026
Error = :4.23e-03; Runtime: 1871.4s; Iterations = 820, grads =   3.419e-06, unitary_metric = 1.00025
Error = :4.05e-03; Runtime: 1918.9s; Iterations = 840, grads =   3.342e-06, unitary_metric = 1.00024
Error = :3.86e-03; Runtime: 1966.5s; Iterations = 860, grads =   3.212e-06, unitary_metric = 1.00023
Error = :3.66e-03; Runtime: 2014.1s; Iterations = 880, grads =   3.028e-06, unitary_metric = 1.00024
Error = :3.48e-03; Runtime: 2061.6s; Iterations = 900, grads =   2.830e-06, unitary_metric = 1.00024
Error = :3.33e-03; Runtime: 2112.5s; Iterations = 920, grads =   2.586e-06, unitary_metric = 1.00023
Error = :3.16e-03; Runtime: 2160.2s; Iterations = 940, grads =   2.345e-06, unitary_metric 

Error = :3.92e-02; Runtime: 983.7s; Iterations = 480, grads =   3.067e-04, unitary_metric = 1.00052
Error = :3.92e-02; Runtime: 1023.4s; Iterations = 500, grads =   1.315e-04, unitary_metric = 1.00052
Error = :3.92e-02; Runtime: 1066.6s; Iterations = 520, grads =   1.633e-05, unitary_metric = 1.00053
Error = :3.92e-02; Runtime: 1106.7s; Iterations = 540, grads =   9.361e-06, unitary_metric = 1.00053
Error = :3.91e-02; Runtime: 1146.7s; Iterations = 560, grads =   2.780e-05, unitary_metric = 1.00053
Error = :3.91e-02; Runtime: 1186.8s; Iterations = 580, grads =   1.136e-03, unitary_metric = 1.00053
Error = :4.04e-02; Runtime: 1226.8s; Iterations = 600, grads =   1.111e-01, unitary_metric = 1.00052
Error = :3.94e-02; Runtime: 1270.0s; Iterations = 620, grads =   2.156e-02, unitary_metric = 1.00053
Error = :3.91e-02; Runtime: 1310.0s; Iterations = 640, grads =   1.869e-03, unitary_metric = 1.00053
Error = :3.91e-02; Runtime: 1349.9s; Iterations = 660, grads =   1.590e-04, unitary_metric =

Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.95e-01; Runtime: 4.5s; Iterations = 0, grads =   1.199e-02, unitary_metric = 1.00060
Error = :2.12e-01; Runtime: 54.6s; Iterations = 20, grads =   8.950e+00, unitary_metric = 1.00041
Error = :5.68e-02; Runtime: 97.6s; Iterations = 40, grads =   1.047e+00, unitary_metric = 1.00042
Error = :4.37e-02; Runtime: 140.6s; Iterations = 60, grads =   1.990e-01, unitary_metric = 1.00045
Error = :4.10e-02; Runtime: 183.6s; Iterations = 80, grads =   3.240e-02, unitary_metric = 1.00043
Error = :4.05e-02; Runtime: 226.6s; Iterations = 100, grads =   2.873e-03, unitary_metric = 1.00043
Error = :4.05e-02; Runtime: 272.6s; Iterations = 120, grads =   4.745e-04, unitary_metric = 1.00043
Error = :4.04e-02; Runtime: 315.5s; Iterations = 140, grads =   5.996e-05, unitary_metric = 1.00043
Error = :4.04e-02; Runtime: 358.4s; Iterations = 160, grads =   1.531e-05, unitary_metric =

Error = :3.97e-02; Runtime: 3502.9s; Iterations = 1600, grads =   1.873e-08, unitary_metric = 1.00044
Error = :3.97e-02; Runtime: 3548.7s; Iterations = 1620, grads =   1.874e-08, unitary_metric = 1.00045
Error = :3.97e-02; Runtime: 3591.7s; Iterations = 1640, grads =   1.920e-08, unitary_metric = 1.00044
Error = :3.97e-02; Runtime: 3634.5s; Iterations = 1660, grads =   1.956e-08, unitary_metric = 1.00044
Error = :3.97e-02; Runtime: 3677.3s; Iterations = 1680, grads =   1.910e-08, unitary_metric = 1.00044
Error = :3.97e-02; Runtime: 3720.3s; Iterations = 1700, grads =   1.989e-08, unitary_metric = 1.00045
Error = :3.97e-02; Runtime: 3766.2s; Iterations = 1720, grads =   1.995e-08, unitary_metric = 1.00044
Error = :3.96e-02; Runtime: 3808.9s; Iterations = 1740, grads =   2.024e-08, unitary_metric = 1.00045
Error = :3.96e-02; Runtime: 3851.6s; Iterations = 1760, grads =   1.988e-08, unitary_metric = 1.00045
Error = :3.96e-02; Runtime: 3894.4s; Iterations = 1780, grads =   2.040e-08, unita

In [ ]:
print(pulse_times)

## 3Reg Gate-Based

In [4]:
print('Times reflect unoptimized and unmapped -> after optimization (time goes down) --> after mapping (time goes up)')
unitaries = []
gate_times = []
for benchmark, N, layout, p in [('3Reg', 4, (2, 2), 1),
                              ('3Reg', 4, (2, 2), 2),
                              ('3Reg', 4, (2, 2), 3),
                              ('3Reg', 4, (2, 2), 4),
                              ('3Reg', 4, (2, 2), 5),
                              ('3Reg', 4, (2, 2), 6),
                              ('3Reg', 6, (2, 3), 1),
                              ('3Reg', 6, (2, 3), 2),
                              ('3Reg', 6, (2, 3), 3),
                              ('3Reg', 6, (2, 3), 4),
                              ('3Reg', 6, (2, 3), 5),
                              ('3Reg', 6, (2, 3), 6)]:
    unoptimized_unmapped_time = 0.0
    unmapped_time = 0.0
    mapped_time = 0.0
    samples = 10
    for _ in range(samples):
        circuit = qaoa.get_qaoa_circuit(N, p, benchmark)
        unoptimized_unmapped_time += np.round(util.circuitutil.get_max_pulse_time(circuit), 4)
        circuit = util.circuitutil.optimize_circuit(circuit)
        unmapped_time += np.round(util.circuitutil.get_max_pulse_time(circuit), 4)
        coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(layout[0], layout[1])
        circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)
        mapped_time += np.round(util.circuitutil.get_max_pulse_time(circuit), 4)
    template = 'benchmark: %s (N=%s, p=%s). Times %s --> optimized = %s --> mapped = %s'
    print(template % (benchmark, N, p, np.round(unoptimized_unmapped_time/samples,4), np.round(unmapped_time/samples,4), np.round(mapped_time/samples,4)))
    gate_times.append(np.round(mapped_time/samples,4))

Times reflect unoptimized and unmapped -> after optimization (time goes down) --> after mapping (time goes up)
benchmark: 3Reg (N=4, p=1). Times 51.9 --> optimized = 51.9 --> mapped = 76.32
benchmark: 3Reg (N=4, p=2). Times 99.1 --> optimized = 99.1 --> mapped = 144.98
benchmark: 3Reg (N=4, p=3). Times 146.3 --> optimized = 146.3 --> mapped = 220.3
benchmark: 3Reg (N=4, p=4). Times 193.5 --> optimized = 193.5 --> mapped = 291.18
benchmark: 3Reg (N=4, p=5). Times 240.7 --> optimized = 240.7 --> mapped = 342.08
benchmark: 3Reg (N=4, p=6). Times 287.9 --> optimized = 287.9 --> mapped = 416.66
benchmark: 3Reg (N=6, p=1). Times 54.3 --> optimized = 54.3 --> mapped = 118.11
benchmark: 3Reg (N=6, p=2). Times 111.4 --> optimized = 111.4 --> mapped = 230.25
benchmark: 3Reg (N=6, p=3). Times 165.26 --> optimized = 165.26 --> mapped = 326.7
benchmark: 3Reg (N=6, p=4). Times 227.25 --> optimized = 227.25 --> mapped = 391.0
benchmark: 3Reg (N=6, p=5). Times 263.02 --> optimized = 263.02 --> mapped 

In [5]:
print(gate_times)

[76.32, 144.98, 220.3, 291.18, 342.08, 416.66, 118.11, 230.25, 326.7, 391.0, 488.75, 616.48]
